## __[DESAFIO #2] Técnicas para el procesamiento del lenguaje (NLP + LLMs)__
Se desea automatizar y elevar el nivel de precisión del proceso de revisión de perfiles de hojas de vida (CV). Para cumplir con este objetico se debe implementar un modelo IA basado en técnicas NLP, LLMs, o una combinación de algunas de estas.
El proceso de revisión busca obtener la siguiente información (especificaciones técnicas del perfil), de cada hoja de vida:

    - Nombre completo del candidato
    - Email o teléfono de contacto
    - Número total de años de experiencia profesional
    - ¿Tiene formación en inteligencia artificial? (S/N)

### PRUEBA TÉCNICA Databricks MLOps + IA

El modelo IA deberá retornar un conjunto de datos en formato JSON, con el resultado de la validación, el cual debe contener los valores obtenidos por cada especificación técnica, incluyendo un valor de score que indicará el nivel de precisión o el porcentaje de ajuste del valor obtenido por cada especificación técnica. Es libre de definir el formato del JSON resultante.

En el ZIP adjunto encontrará una carpeta “CVs” con una muestra de hojas de vida.

En los casos, donde el modelo IA no pueda obtener un valor, deberá registrar un valor nulo con un score de cero (0).
Sugerencias:

    - Trabaje en un modelo IA prototipo, que sirva como prueba de concepto. Por el tiempo que exige la prueba, no esperamos que nos entregue un modelo perfecto. Nos interesa el enfoque de la solución y una prueba de concepto verificable.

    - El lenguaje de programación a usar debe ser Python. Siéntase libre de incorporar las librerías que considere a bien.

    - Considere desarrollar el ejercicio usando un Notebook de Google Colab. Publique el Notebook en el repo de GitHub. Si incluye un botón de enlace o un link al Notebook en Google Colab, por favor compartirlo a la cuenta jorgegr79@gmail.com

    - La documentación de la solución será un aspecto que consideraremos. Puede realizar un “readme” markdown en GitHub, publicar un PDF, desarrollar un notebook, o cualquier otra alternativa que considere a bien.

    - Siéntase libre de registrar los errores que cometa en el proceso de implementación; entendemos y aceptamos la incertidumbre y vemos oportunidades

cuando los errores son comprendidos y superados. Piense en la deuda técnica y hazla explícita en la documentación.
Entregable: Publicar todos los artefactos desarrollados en un repo de GitHub. Incluir la documentación de lo realizado. Se valorará el paso-a-paso y las capturas
de pantalla con las evidencias de las pruebas realizadas. Si le resulta más sencillo sustituir la documentación por un video explicativo, está muy bien, entonces no olvide publicar enlace al video y asegurarse que quede accesible.

In [1]:
# !pip install pdfminer.six

### Librerias

In [2]:
import os
import pickle
import re
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from pdfminer.high_level import extract_text
from transformers import (
    AutoModelForQuestionAnswering,
    # AutoModelForSeq2SeqLM,
    AutoModelForTokenClassification,
    # AutoModelForSequenceClassification
    AutoTokenizer,
    pipeline,
)


/home/bdebian/.virtualenvs/mvm_test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Funciones de apoyo

In [3]:
def descargar_archivo_drive(id_archivo:str, nombre_archivo:str)-> None:
    """
    Descarga un archivo desde Google Drive usando el enlace público.

    Args:
        url: La URL pública del archivo en Google Drive.
        nombre_archivo: El nombre con el que quieres guardar el archivo.
    """

    # URL de descarga directa
    url_descarga = f'https://drive.google.com/uc?export=download&id={id_archivo}'

    # Hacer la petición HTTP para descargar el archivo
    respuesta = requests.get(url_descarga)

    # Guardar el archivo
    with open(nombre_archivo, 'wb') as f:
        f.write(respuesta.content)

    print(f"Archivo '{nombre_archivo}' descargado con éxito.")

def extract_text_from_pdf(file_path:str)->str:
    return extract_text(file_path)

In [4]:

data = {
    "cv1":'1b5jdLO1tRcam2n8W7GowHxbbrhS9_HBU',
    "cv2":'1mvEslp2nQWn_yIoqEInkIHiN2ptGJ-OA',
    "cv3":'1r2rumnNFbuk_ziF5zaMouys3hHIr2LDj',
    "cv4":'1E4lI9zH2dUmhvX2JXtCBZYIVB-TYdfQr',
    "cv5":'1Vn1F4RNbj_d4SynnMYBC4DRq2K73zPEv',
    "cv6":'1uy7J5FGmd29gwNsrUbdyxIgT2a-LX6xt',
    "cv7":'1jv3RdkeV_5iP3UE8RpW-xlZffLTfYaph',
    "cv8":'1f4gsa7QjqO_HiGbT6AQMRuLPjtdEUhlS',
    "cv9":'1ry_hPo-Rrsx4sv1XUa8OC667a10YtUlR',
    }

for key, value in data.items():
  filename = Path(key).with_suffix('.pdf')
  if not filename.is_file():
    descargar_archivo_drive(value, filename.as_posix())


 El proceso de revisión busca obtener la siguiente información (especificaciones técnicas del perfil), de cada hoja de vida:

- Nombre completo del candidato
- Email o teléfono de contacto
- Número total de años de experiencia profesional
- ¿Tiene formación en inteligencia artificial? (S/N)

Cargar documento aleatorio

In [5]:
# Seleccion aleatoria de archivo
dict_files = list(data.keys())
dict_file = dict_files[np.random.randint(0, len(dict_files))]
filename = Path(dict_file).with_suffix('.pdf')

# for key, value in data.items():
#   filename = Path(key).with_suffix('.pdf')
print("\n Archivo:",filename.as_posix(),"\n")
text = extract_text_from_pdf(filename.as_posix())
print(f" Longitud de caracteres {len(text)}")
print(text)


 Archivo: cv7.pdf 

 Longitud de caracteres 4896
Ringgi Cahyo Dwiputra 

South Tangerang, Banten • ringgicahyo@gmail.com • 085157115062 • linkedin.com/in/ringgicahyo/  

Education 

Universitas Indonesia   
Bachelor of Computer Science, Majoring in Information System – GPA: 3.62 

       Depok, West Java
  Sep 2017 – Sep 2021 

Professional Experiences 

PT Kita Lulus Internasional (EduTech, Seed Funding, 200k+ Users)   
Associate Product Manager 

       West Jakarta, DKI Jakarta 
     Feb 2021 – Present 
•  Managing the whole mobile application products, contributing to gaining over 100k installations with a 

DAU to MAU ratio of 20% and increasing the average engagement time per session by 150% 
•  Developing a job marketplace platform for low-to-mid job levels that reaches more than 46k job 

applications with an average of 2k+ job applications per day in less than 2 months 

•  Executing a community feature to connect between job seekers, resulting in more than 11k community 

me

El modelo IA deberá retornar un conjunto de datos en formato JSON, con el resultado de la validación.

El cual debe contener los valores obtenidos por:

 - Cada especificación técnica
 - Incluyendo un valor de score que indicará el nivel de precisión o el porcentaje de ajuste del valor obtenido por cada especificación técnica
 - Es libre de definir el formato del JSON resultante.

## Procesamiento

### Cargar modelos


El código `Model` define una clase para cargar y guardar modelos preentrenados de procesamiento de lenguaje natural (NLP) utilizando Hugging Face Transformers. Estos modelos se utilizan para tareas de respuesta a preguntas (QA), reconocimiento de entidades nombradas (NER) y clasificación de cero-shot.

**Funcionamiento:**

1. **Inicialización:**
   * Se define un diccionario `model_list_names` con los nombres de los modelos preentrenados para diferentes tareas.
   * Se crea un diccionario `t_model` para almacenar los modelos cargados.

2. **Serialización:**
   * La función `pickle_model` guarda un objeto en un archivo pickle para su uso posterior.
3. **Deserialización:**
   * La función `load_from_memory` carga un objeto desde un archivo pickle.
4. **Carga de Modelos Preentrenados:**
   * La función `load_model_pretrain` carga un modelo preentrenado utilizando `pipeline` de Hugging Face Transformers. Si el modelo no existe localmente, se carga desde Hugging Face y se guarda en un archivo pickle.
5. **Obtención de Modelos:**
   * La función `get_qa_model` verifica si el modelo existe localmente y lo carga desde el archivo pickle. Si no existe, lo carga desde Hugging Face y lo guarda.

In [6]:
class Model:
    """
    This Python class defines methods for pickling and loading pre-trained question-answering models
    using Hugging Face Transformers."""

    def __init__(self):
        self.model_list_names = {
            "question-answering": 'distilbert-base-cased-distilled-squad', #'distilbert-base-cased-distilled-squad' ,"deepset/roberta-base-squad2"
            "ner": "Jean-Baptiste/camembert-ner-with-dates", #"dslim/bert-base-NER",
            'zero-shot-classification':"facebook/bart-large-mnli" #'facebook/bart-large-mnli
        }
        self.t_model = {
            'ner':AutoModelForTokenClassification.from_pretrained(self.model_list_names['ner']),
            'question-answering':AutoModelForQuestionAnswering.from_pretrained(self.model_list_names['question-answering'])

            }

    def pickle_model(self, obj, file_name):
        """
        The function `pickle_model` saves an object to a file using pickle serialization in Python.

        Args:
          obj: The `obj` parameter in the `pickle_model` function refers to the object that you want to
        serialize and save to a file using the `pickle` module in Python. This object can be any Python
        object such as a model, dictionary, list, etc.
          file_name: The `file_name` parameter is a string that represents the name of the file where the
        object will be saved after pickling.
        """
        with open(f"{file_name}.pickle", "wb") as f:
            pickle.dump(obj, f)

    def load_from_memory(self, file_name):
        """
        The function `load_from_memory` reads and loads a pickled file from memory.

        Args:
          file_name: The `file_name` parameter in the `load_from_memory` function is a string that
        represents the name of the file from which you want to load data. The function opens a file with the
        name `{file_name}.pickle` in binary read mode and then loads the data from that file using the

        Returns:
          The `load_from_memory` method is returning the data loaded from the pickle file specified by the
        `file_name` parameter.
        """
        with open(f"{file_name}.pickle", "rb") as f:
            return pickle.load(f)

    def load_model_pretrain(self, model_name: str, type_model: str):
        """
        This function loads a pre-trained model for question answering using the specified model name and
        type.

        Args:
          model_name (str): The `model_name` parameter is a string that represents the name or identifier of
        the model being loaded or pretrained. It is used as a reference to save the model object for future
        use.
          type_model (str): The `type_model` parameter is the name of the pre-trained model that will be
        loaded using the Hugging Face `AutoTokenizer` and `AutoModelForQuestionAnswering` classes. This
        model will be used for question answering tasks in the subsequent code.
        """
        # print(model_name,type_model)
        if model_name == 'zero-shot-classification':
          nlp = pipeline(model_name,
                         model=type_model,
                        device=0,
                         )
          self.pickle_model(obj=nlp, file_name=model_name)
        else:
          tokenizer = AutoTokenizer.from_pretrained(type_model)
          nlp = pipeline(model_name,
                        model=self.t_model[model_name],
                        tokenizer=tokenizer,
                        device=0,
                        aggregation_strategy="simple",
                        grouped_entities=True
                        )
          self.pickle_model(obj=nlp, file_name=model_name)

    def get_qa_model(self, name:str):
        """
        The function `get_qa_model` loads a pre-trained model and returns it from memory.

        Args:
          name: The `name` parameter in the `get_qa_model` method is used to specify the name of the
        question-answering model that you want to retrieve. This name is then used to load the pre-trained
        model and retrieve it from memory.

        Returns:
          The `get_qa_model` function returns the model loaded from memory with the specified name.
        """
        if Path(f"{name}.pickle").is_file():
            return self.load_from_memory(file_name=name)
        else:
          self.load_model_pretrain(name, self.model_list_names[name])
          return self.load_from_memory(file_name=name)

        # self.load_model_pretrain(name, self.model_list_names[name])
        # return self.load_from_memory(file_name=name)

# load_model = Model()
# modeltest_qa = load_model.get_qa_model("zero-shot-classification")
# # modeltest_qa = load_model.get_qa_model('question-answering')
# # modeltest = load_model.get_qa_model('ner')


### Extracion de texto
**Propósito:**

El código `ParcerCV` está diseñado para extraer información relevante de un texto de currículum vitae (CV), como el nombre de la persona, su correo electrónico, número de teléfono, experiencia laboral y habilidades relacionadas con la inteligencia artificial (IA).

**Funcionamiento:**

1. **Inicialización:**
   * Se crea un objeto `ParcerCV` con parámetros opcionales para el modelo y el texto del CV.
   * Se definen prompts y expresiones regulares para extraer información específica (nombre, correo electrónico, número de teléfono, experiencia y habilidades de IA).
   * Se crea una lista de habilidades relacionadas con la IA.

2. **Modelos de Lenguaje:**
   * Se utiliza un modelo de lenguaje para responder preguntas (QA) y otro para reconocimiento de entidades nombradas (NER).

3. **Extracción de Información:**
   * **Datos personalizados:** Se utilizan expresiones regulares para extraer datos como correo electrónico y número de teléfono.
   * **Preguntas y respuestas:** Se utiliza el modelo de QA para responder preguntas sobre información específica del CV (nombre, experiencia).
   * **Habilidades de IA:** Se utiliza el modelo de NER para identificar habilidades relacionadas con la IA en el texto.

4. **Clasificación de Habilidades:**
   * Se utiliza un modelo de clasificación de cero-shot para clasificar las habilidades encontradas y calcular una puntuación de validación.

**Retorno:**

El código devuelve información extraída del CV, incluyendo:

* Nombre de la persona
* Correo electrónico
* Número de teléfono
* Experiencia laboral (en años)
* Habilidades de IA (sí/no)
* Puntuación de validación de habilidades


In [7]:
class ParcerCV:
    """
    # The `ParcerCV` class is designed to extract information from a resume text, including the person's
    # name, email, phone number, experience, and AI skills.
    """
    def __init__(self, **arg):
        self.model = arg.get("model", None)
        self.text = arg.get("text", None)
        self.user_data_promt = {
            "name": "What is the person's name?",
            "email": "What is the person's email?",
            "phone": "What is the person's phonenumber?",
            "experience": "How many years of experience does the person have?",
            "ai": "Does the person have AI"
        }
        self.user_data_re = {
            "email": re.compile(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+"),
            "phone": re.compile(r"[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]\s{1,4}"),
        }
        self.skill_dc = [
          'machine learning',
          'data science',
          'python',
          'tensorflow',
          'keras',
          'pytorch',
          'scikit-learn',
          'nlp',
          'deep learning',
          'computer vision',
          'natural language processing',
          'dask',
          'numpy',
          'pandas',
          'matplotlib',
          'seaborn',
          'spacy',
          ]


    def qa_model(self, promt: str, text: str):
        """
        The function `extract_info` takes a prompt and text as input, creates a prompt context dictionary,
        retrieves a question-answering model, and returns the model's response to the prompt context.

        Args:
          promt (str): The `promt` parameter is a string that represents the question or prompt for which
        you want to extract information from the given `text`. It serves as the query or input for the
        question-answering model to find relevant information in the provided text.
          text (str): The `text` parameter in the `extract_info` function is the text from which you want
        to extract information based on the prompt provided. The function creates a context dictionary with
        the prompt and the text, then uses a question-answering model to extract information from this
        context.

        Returns:
          The `extract_info` function takes in a prompt and text as input parameters. It creates a
        dictionary `promt_context` with the keys "question" and "context" set to the input prompt and text
        respectively. It then uses a question-answering model obtained from
        `self.model.get_qa_model("question-answering")` to find the answer to the question in the
        """
        promt_context = {"question": promt, "context": text}
        qa_model = self.model.get_qa_model("question-answering")
        return qa_model(promt_context)

    def ner_model(self,text: str):
        qa_model = self.model.get_qa_model("ner")
        return qa_model(text)

    def extract_custom_data(self, resume_text: str, typo_re: re.Pattern):
        """
        The `extract_custom_data` function takes in a resume text and a regular expression pattern, and
        returns a list of all matches of the pattern in the resume text.

        Args:
          resume_text (str): The `resume_text` parameter is a string that contains the text of a resume. It
        is the input text from which you want to extract custom data using the regular expression pattern
        specified by the `typo_re` parameter.
          typo_re (re.Pattern): The `typo_re` parameter is a regular expression pattern that is used to
        search for specific patterns or sequences of characters within the `resume_text` string. When the
        `extract_custom_data` function is called, it will use this regular expression pattern to extract
        custom data from the provided `resume_text

        Returns:        The `get_email` function extracts and returns the email address from the text using a regular
        expression defined in `user_data_re`.

        Returns:
          The `get_email` method is returning the email address extracted from the text using a regular
        expression pattern defined in `user_data_re["email"]`.
          The `extract_custom_data` method is returning a list of all non-overlapping matches of the regular
        expression pattern `typo_re` found in the `resume_text` string.
        """
        return re.findall(typo_re, resume_text)

    def get_email(self):
        """
        The `get_email` function extracts and returns the email address from the text using a regular
        expression defined in `user_data_re`.

        Returns:
          The `get_email` method is returning the email address extracted from the text using a regular
        expression pattern defined in `user_data_re["email"]`.
        """
        return self.extract_custom_data(self.text, self.user_data_re["email"])[0]

    def get_phonenumber(self):
        """
        This function retrieves the phone number from a given text using a regular expression pattern stored
        in user data.

        Returns:
          The `get_phonenumber` method is returning the phone number extracted from the text using the
        regular expression pattern defined in `self.user_data_re["phone"]`.
        """
        return self.extract_custom_data(self.text, self.user_data_re["phone"])[0]

    def get_max_entities(sefl, entities: list) -> defaultdict:
        """
        The function `get_max_entities` takes a list of entities and returns a dictionary containing the
        entity with the highest score for each entity group.

        Args:
          sefl: It looks like there is a typo in the function definition. The parameter "sefl" should be
        "self" instead. The corrected function definition should be:
          entities (list): The `entities` parameter is a list of dictionaries where each dictionary
        represents an entity with keys "entity_group" and "score". The function `get_max_entities` iterates
        over these entities to find the maximum score for each unique "entity_group" and stores the entity
        with the highest score in the

        Returns:
          The function `get_max_entities` returns a dictionary containing the maximum score for each entity
        group based on the input list of entities.
        """
        # Diccionario para almacenar el máximo score para cada entity_group
        max_entities = defaultdict(lambda: {"score": 0})

        for entity in entities:
            group = entity["entity_group"]
            # Si la puntuación actual es mayor que la almacenada, actualiza
            if entity["score"] > max_entities[group]["score"]:
                max_entities[group] = entity
        return max_entities

    def filter_entities(sefl, entities: list) -> list:
        """
        The `filter_entities` function takes a list of entities and filters them based on specific criteria
        for entity groups and scores.

        Args:
          sefl: It looks like there is a typo in the function definition. The parameter "sefl" should be
        "self" instead. The corrected function definition should be:
          entities (list): The `filter_entities` function you provided seems to filter a list of entities
        based on certain criteria related to their "entity_group" and "score" attributes. However, it seems
        like there might be a typo in the function definition where "sefl" should be "self".

        Returns:
          The `filter_entities` function returns a list of entities that meet certain criteria based on
        their "entity_group" and "score" values. The function filters the input list of entities based on
        the following conditions:
        """
        filtered_entities = []
        for entity in entities:
            if entity["entity_group"] == "PER" and entity["score"] > 0.9:
                # Regla para nombres de personas
                filtered_entities.append(entity)
            elif entity["entity_group"] == "ORG" and entity["score"] > 0.7:
                # Reglas para organizaciones
                filtered_entities.append(entity)
            # Otras reglas para diferentes tipos de entidades
        return filtered_entities


    def classify_skills(self):
        """
        This function classifies skills based on a zero-shot classification model and calculates a
        validation score.

        Returns:
          The `classify_skills` method returns two values: `exp_ai` and `val_score`.
        """
        classifier = self.model.get_qa_model("zero-shot-classification")
        results = classifier(self.text, self.skill_dc)

        detected_skills = [
            (label, score)
            for label, score in zip(results["labels"], results["scores"])
            if score > 0.1
        ]
        features_found = len(detected_skills)
        if features_found != 0:
          exp_ai = "yes"
          val_score = features_found/len(self.skill_dc)
        else:
          exp_ai= "no"
          val_score = 0

        return exp_ai, val_score, detected_skills

Uso de la librería para extraer información donde se instancia la clase `ParserCV'

In [8]:
eval_cv_text = {
    'text':text,
    'model':Model()
    }
parcer = ParcerCV(**eval_cv_text)

Extracción de nombre de usuario

Usando Métodos de la clase `ParceCV` para utilizar los modelos __*ner*__, para filtrarse por las entidades  encontradas con `filter_entities` en esta caso el nombre de la persona, para después obtener su máximo valores con la función `get_max_entities`

En caso de no encontrar un respuesta valida usara el modelo de  `question-answering`, donde usando un promt con la pregunta del nombre en el texto del CV

In [9]:
entities = parcer.ner_model(text = parcer.text)
filtered_entities = parcer.filter_entities(entities)
extract_entities = parcer.get_max_entities(filtered_entities)

try:
    name = extract_entities["PER"]["word"]
    score_name = extract_entities["PER"]["score"]
except:
    name_search = parcer.qa_model(promt=parcer.user_data_promt["name"], text=parcer.text)
    name= name_search["answer"]
    score_name = name_search["score"]

/home/bdebian/.virtualenvs/mvm_test/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Extracción de correo electrónico o número celular

Esto hace con la función `extract_custom_data` que tiene asociada una expresión regular para encontrar el patrón en el texto correspondiente al número celular o email en el CV

In [10]:
email_parce = parcer.extract_custom_data(text, parcer.user_data_re["email"])
if len(email_parce) >=  1:
    email_parce = email_parce[0]
else:
    email_parce = []

phone_parce = parcer.extract_custom_data(text, parcer.user_data_re["phone"])

if len(phone_parce) >= 1:
    phone_parce = phone_parce[0]
else:
    phone_parce = []

Extracción si tiene formación en IA

`classify_skills` es una función que utiliza un modelo de clasificación de cero-shot para identificar habilidades relacionadas con la inteligencia artificial en un texto dado. Además, calcula una puntuación de validación para evaluar la calidad de la clasificación.

Se pasa internamente el texto del curriculum vitae y una lista de habilidades al modelo de clasificación, el modelo devuelve una lista de etiquetas y sus correspondientes puntuaciones.
Se calcula la proporción de habilidades detectadas con respecto al total de habilidades definidas

In [11]:
exp_ia,score_ia,skills = parcer.classify_skills()

Extracción de años de experiencia 
para esto es emplea el modelo  `question-answering` para introducir un promt con su respectivo contexto preguntado por la experiencia en el curriculum vitae


In [12]:
name_search = parcer.qa_model(
    promt=parcer.user_data_promt["experience"], text=parcer.text
)
exp_year = name_search["answer"]
score_year = name_search["score"]
print(exp_year, score_year)

13 0.003106752410531044



El código crea un diccionario llamado `data_parced` para almacenar información extraída de un currículum vitae (CV). Luego, guarda este diccionario en un archivo JSON.

**Estructura del Diccionario:**

El diccionario `data_parced` contiene las siguientes claves y sus valores correspondientes:

* **name:**
  * **value:** Contiene el nombre de la persona.
  * **score:** Contiene una puntuación asociada al nombre (posiblemente relacionada con la confianza en la extracción).
* **email:**
  * **value:** Contiene la dirección de correo electrónico.
  * **score:** Se establece como "NA" ya que no se proporciona una puntuación para el correo electrónico.
* **phone:**
  * **value:** Contiene el número de teléfono.
  * **score:** Se establece como "NA" ya que no se proporciona una puntuación para el número de teléfono.
* **ai:**
  * **value:** Contiene un valor booleano indicando si la persona tiene experiencia en IA.
  * **score:** Contiene la puntuación de validación para la clasificación de habilidades de IA.
* **experience:**
  * **value:** Contiene el número de años de experiencia.
  * **score:** Se establece como "NA" ya que no se proporciona una puntuación para la experiencia.

**Guardado en JSON:**

El diccionario `data_parced` se guarda en un archivo JSON llamado "parce_cv.json" con sangría de 4 espacios para una mejor legibilidad.


In [13]:
data_parced = {
    "name": {
        "value": name,
        "score": str(score_name),
    },
    "email": {
        "value": email_parce,
        "score": "NA",
    },
    "phone": {
        "value": phone_parce,
        "score": "NA",
    },
    "ai": {"value": exp_ia, "score": str(score_ia)},
    "experience": {"value": exp_year, "score": str(score_year)},
}


import json

with open("parce_cv.json", "w") as fp:
    json.dump(data_parced, fp, indent=4)
